In [ ]:
import numpy as np
import cv2
import tensorflow as tf
import os

# Load TFLite model
interpreter = tf.lite.Interpreter(model_path="yolov5n-fp16.tflite")
interpreter.allocate_tensors()

# Get model input and output details
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Load and preprocess image
def preprocess_image(img_path, input_shape):
    image = cv2.imread(img_path)
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    resized = cv2.resize(image_rgb, input_shape)
    input_data = np.expand_dims(resized / 255.0, axis=0).astype(np.float32)
    return image, input_data

# Post-processing (YOLO-style decoding)
def decode_output(output, conf_threshold=0.25):
    # [N, 6] = x, y, w, h, confidence, class
    boxes = []
    for det in output[0]:  # Assuming output[0] is [N, 6]
        if det[4] > conf_threshold:
            x_center, y_center, width, height = det[0:4]
            x1 = int(x_center - width / 2)
            y1 = int(y_center - height / 2)
            x2 = int(x_center + width / 2)
            y2 = int(y_center + height / 2)
            confidence = float(det[4])
            class_id = int(det[5])
            boxes.append((x1, y1, x2, y2, confidence, class_id))
    return boxes

# Draw detections
def draw_boxes(image, boxes, class_names):
    for x1, y1, x2, y2, conf, cls in boxes:
        label = f"{class_names[cls]}: {conf:.2f}"
        cv2.rectangle(image, (x1, y1), (x2, y2), (0, 255, 0), 2)
        cv2.putText(image, label, (x1, y1 - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

# --- Main execution ---
# Update this with your own image path and classes
IMAGE_PATH = "test_image_vids/test_image_1.jpg"
CLASS_NAMES = ["person", "bicycle", "car", "motorbike", "aeroplane", "bus", "train"]  # Example names

# Step 1: Check if file exists
if not os.path.exists(IMAGE_PATH):
    print(f"❌ File does NOT exist at: {IMAGE_PATH}")
else:
    print(f"✅ File exists at: {IMAGE_PATH}")

    # Step 2: Try to load with OpenCV
    image = cv2.imread(IMAGE_PATH)
    
    if image is None:
        print("❌ OpenCV failed to load the image. It might be corrupted or unreadable.")
    else:
        print("✅ OpenCV successfully loaded the image.")
        
        # Resize for display (width = 640, keep aspect ratio)
        display_width = 640
        h, w = image.shape[:2]
        scale = display_width / w
        new_dim = (display_width, int(h * scale))
        resized_display = cv2.resize(image, new_dim)

        cv2.imshow("Test Image", resized_display)
        cv2.waitKey(0)
        cv2.destroyAllWindows()


✅ File exists at: test_image_vids/test_image_1.jpg
✅ OpenCV successfully loaded the image.


In [3]:
%pwd

'c:\\Users\\Kirk Recio\\Documents\\PYTHON\\WearableAI_OrangePi'

In [ ]:
import numpy as np
import cv2
import tensorflow as tf
import os

# Load TFLite model
interpreter = tf.lite.Interpreter(model_path="yolov5n-fp16.tflite")
interpreter.allocate_tensors()

# Get model input and output details
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Preprocess image
def preprocess_image(img_path, input_shape):
    image = cv2.imread(img_path)
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    resized = cv2.resize(image_rgb, input_shape)
    input_data = np.expand_dims(resized / 255.0, axis=0).astype(np.float32)
    return image, input_data

# Post-process output
def decode_output(output, conf_threshold=0.25):
    boxes = []
    for det in output[0]:  # [N, 6] format
        if det[4] > conf_threshold:
            x_center, y_center, width, height = det[0:4]
            x1 = int(x_center - width / 2)
            y1 = int(y_center - height / 2)
            x2 = int(x_center + width / 2)
            y2 = int(y_center + height / 2)
            confidence = float(det[4])
            class_id = int(det[5])
            boxes.append((x1, y1, x2, y2, confidence, class_id))
    return boxes

# Draw bounding boxes
def draw_boxes(image, boxes, class_names):
    for x1, y1, x2, y2, conf, cls in boxes:
        label = f"{class_names[cls]}: {conf:.2f}"
        cv2.rectangle(image, (x1, y1), (x2, y2), (0, 255, 0), 2)
        cv2.putText(image, label, (x1, y1 - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

# Sample class names (replace with your actual .names file or list)
class_names = ["person", "bicycle", "car", "motorcycle", "airplane",
               "bus", "train", "truck", "boat", "traffic light"]

# --- Main Execution ---
IMAGE_PATH = "test_image_vids/test_image_2.jpg"

# Step 1: Check if image exists
if not os.path.exists(IMAGE_PATH):
    print(f"❌ File does NOT exist at: {IMAGE_PATH}")
else:
    print(f"✅ File exists at: {IMAGE_PATH}")
    
    # Step 2: Preprocess image
    original_img, input_tensor = preprocess_image(IMAGE_PATH, (input_details[0]['shape'][2], input_details[0]['shape'][1]))

    # Step 3: Set tensor and run inference
    interpreter.set_tensor(input_details[0]['index'], input_tensor)
    interpreter.invoke()

    # Step 4: Get and decode output
    output_data = interpreter.get_tensor(output_details[0]['index'])
    boxes = decode_output(output_data, conf_threshold=0.3)

    # Step 5: Draw results
    draw_boxes(original_img, boxes, class_names)

    # Step 6: Show the image with detections
    cv2.imshow("Detections", original_img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

✅ File exists at: test_image_vids/test_image_2.jpg
